In [1]:
import os
import requests
import re
from datetime import datetime, timezone
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pyperclip
import time

#git add . && git commit -m "initial commit" && git push origin main

endpoint = "https://active-monitor-48.hasura.app/v1/graphql"
admin_key = "bAQuK7HSYvMAp6S6pnqXH0wQlyuKNUICzoW3jwecc27pwz6COLhE750s5YAec7Hz"

def query_hasura_graphql(endpoint, admin_key, query, variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }

    data = {
        'query': query,
        'variables': variables
    }
    response = requests.post(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)
        
        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False
        
def mutation_hasura_graphql(endpoint, admin_key, mutation_query, mutation_variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }
    response = requests.post(endpoint, json={'query': mutation_query, 'variables': mutation_variables}, headers=headers)
    if response.ok:
        data = response.json()
        print(data)
        return True, data
    else:
        print(f"Mutation failed with status code {response.status_code}: {response.text}")
        return False, None

def update_article_details(offset1):
    graphql_query = '''
    query MyQuery($limit: Int!, $offset: Int!) {
    synopse_articles_t_v1_rss1_articles(limit: $limit, offset: $offset, where: {is_in_detail: {_eq: 0}}, order_by: {post_published: desc}) {
        post_link
        is_default_image
        id
        }
    }
    '''
    offset = offset1
    mutation_query = """
    mutation MyMutation($objects: [synopse_articles_t_v1_rss1_articles_detail_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}) {
        insert_synopse_articles_t_v1_rss1_articles_detail(objects: $objects, on_conflict: {constraint: t_v1_rss1_articles_detail_article_id_key}) {
            affected_rows
        }
        update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
            affected_rows
        }
        }

    """    
    options = webdriver.EdgeOptions()
    options.use_chromium = True
    options.add_experimental_option("prefs", {
        "profile.default_content_setting_values.notifications": 2,  # Disable notifications
        "profile.default_content_setting_values.media_stream_mic": 2,  # Disable microphone
        "profile.default_content_setting_values.media_stream_camera": 2,  # Disable camera
        "profile.default_content_setting_values.geolocation": 2,  # Disable location services
        "profile.default_content_setting_values.automatic_downloads": 2  # Disable automatic downloads
    })
    options.add_argument("--no-first-run")  # Skip the welcome page
    options.add_argument("--disable-infobars")  # Disable the info bars
    options.add_argument("--disable-extensions")  # Disable extensions
    options.add_argument("--disable-popup-blocking")  # Disable popups
    options.add_argument("--disable-features=EdgeTips") 
    options.page_load_strategy = 'eager'
    options.add_argument('--enable-immersive-reader')
    driver = webdriver.Edge(options=options)
    while True:
        variables = {
        "limit": 2,
        "offset": offset1
        }
        response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
        #print(variables, response_data)
        #print(response_data)
        post_links_array = []
        ids=[]
        is_default_image_array = []
        if response_data:
            post_links_array = [item["post_link"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
            is_default_image_array = [item["is_default_image"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
            ids=[item["id"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
        articles_detail = []
        articles_update = []
        if len(post_links_array) == 0:
            break
        try:
            for a in range(len(post_links_array)):
                main_link = post_links_array[a]
                print(main_link)
                driver.get(main_link)
                time.sleep(2)
                get_url = driver.current_url
                read_link= "read://"+get_url
                driver.get(read_link)
                time.sleep(2)
                ActionChains(driver).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()
                ActionChains(driver).key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
                text = pyperclip.paste()
                text2 = text
                text3 = text2.split('\n')
                text3 = [s.replace('\r', '') for s in text3]
                special_chars = set("!@#$%^&*()_+[]{}|;:'\",<>?")
                text4 = [s for s in text3 if len(s) > 0 and (s[0] not in special_chars or s[-1] not in special_chars)]
                my_list = text4
                if my_list[0] == "Hmmm… can't reach this page":
                    offset = offset + 1
                    break
                my_set = set()
                desription = []
                for item in my_list:
                    if item not in my_set:
                        desription.append(item)
                        my_set.add(item)
                #print(desription)
                images_final = []
                articles_detail.append({
                    "article_id": ids[a],
                    "title": desription[0],
                    "description": desription[1:],
                    "image_link": images_final,
                }
                )
                if (is_default_image_array[a] == 0 and len(images_final) > 0):
                    articles_update.append({
                        "where": {"post_link" : { "_eq": main_link }},
                        "_set": {"is_in_detail": 1 , "image_link": images_final[0], "is_default_image": 1}
                    })
                else:
                    articles_update.append({
                        "where": {"post_link" : { "_eq": main_link }},
                        "_set": {"is_in_detail": 1}
                    })
                
                #print(main_link, desription[0], desription[1:], images_final)
            #print(articles_update)
            mutation_variables = {
            "objects": articles_detail,
            "updates": articles_update,
            }
            out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
        except:
            offset = offset + 1
            mutation_variables = {
            "objects": articles_detail,
            "updates": articles_update,
            }
            out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
    driver.quit() 


In [2]:
update_article_details(0)

https://www.nytimes.com/2023/11/17/world/europe/everton-points-deduction.html
https://www.foxnews.com/world/netanyahu-israel-not-successful-minimizing-gaza-civilian-casualties
{'data': {'insert_synopse_articles_t_v1_rss1_articles_detail': {'affected_rows': 2}, 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}, {'affected_rows': 1}]}}
https://www.foxnews.com/world/kremlin-putins-eventual-successor-different-same
https://www.foxnews.com/world/rescue-efforts-india-ramp-up-rescue-40-trapped-workers
{'data': {'insert_synopse_articles_t_v1_rss1_articles_detail': {'affected_rows': 2}, 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}, {'affected_rows': 1}]}}
https://www.foxnews.com/world/iceland-officials-reveal-most-likely-site-possible-volcanic-eruption
https://timesofindia.indiatimes.com/world/south-asia/myanmar-troops-detain-kill-residents-of-rakhine-town-local-media/articleshow/105297691.cms
{'data': {'insert_synopse_articles_t_v1_rss1_articl

In [8]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
synopse_articles_t_v1_rss1_articles(limit: $limit, offset: $offset, where: {is_in_detail: {_eq: 0}}, order_by: {post_published: desc}) {
    post_link
    is_default_image
    id
    }
}
'''
offset1 = 0
mutation_query = """
mutation MyMutation($objects: [synopse_articles_t_v1_rss1_articles_detail_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}) {
    insert_synopse_articles_t_v1_rss1_articles_detail(objects: $objects, on_conflict: {constraint: t_v1_rss1_articles_detail_article_id_key}) {
        affected_rows
    }
    update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
        affected_rows
    }
    }

"""    
options = webdriver.EdgeOptions()
options.use_chromium = True
options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 2,  # Disable notifications
    "profile.default_content_setting_values.media_stream_mic": 2,  # Disable microphone
    "profile.default_content_setting_values.media_stream_camera": 2,  # Disable camera
    "profile.default_content_setting_values.geolocation": 2,  # Disable location services
    "profile.default_content_setting_values.automatic_downloads": 2  # Disable automatic downloads
})
options.add_argument("--no-first-run")  # Skip the welcome page
options.add_argument("--disable-infobars")  # Disable the info bars
options.add_argument("--disable-extensions")  # Disable extensions
options.add_argument("--disable-popup-blocking")  # Disable popups
options.add_argument("--disable-features=EdgeTips") 
options.page_load_strategy = 'eager'
options.add_argument('--enable-immersive-reader')
driver = webdriver.Edge(options=options)
variables = {
"limit": 1,
"offset": offset1
}
response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
#print(variables, response_data)
#print(response_data)
post_links_array = []
ids=[]
is_default_image_array = []
if response_data:
    post_links_array = [item["post_link"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
    is_default_image_array = [item["is_default_image"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
    ids=[item["id"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
articles_detail = []
articles_update = []
for a in range(len(post_links_array)):
    main_link = post_links_array[a]
    print(main_link)
    driver.get(main_link)
    time.sleep(2)
    get_url = driver.current_url
    read_link= "read://"+get_url
    driver.get(read_link)
    time.sleep(2)
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
    text = pyperclip.paste()
    text2 = text
    text3 = text2.split('\n')
    text3 = [s.replace('\r', '') for s in text3]
    special_chars = set("!@#$%^&*()_+[]{}|;:'\",<>?")
    text4 = [s for s in text3 if len(s) > 0 and (s[0] not in special_chars or s[-1] not in special_chars)]
    my_list = text4
    if my_list[0] == "Hmmm… can't reach this page":
        break
    my_set = set()
    desription = []
    for item in my_list:
        if item not in my_set:
            desription.append(item)
            my_set.add(item)
    #print(desription)
    images_final = []
    articles_detail.append({
        "article_id": ids[a],
        "title": desription[0],
        "description": desription[1:],
        "image_link": images_final,
    }
    )
    if (is_default_image_array[a] == 0 and len(images_final) > 0):
        articles_update.append({
            "where": {"post_link" : { "_eq": main_link }},
            "_set": {"is_in_detail": 1 , "image_link": images_final[0], "is_default_image": 1}
        })
    else:
        articles_update.append({
            "where": {"post_link" : { "_eq": main_link }},
            "_set": {"is_in_detail": 1}
        })
    
    #print(main_link, desription[0], desription[1:], images_final)
#print(articles_update)
mutation_variables = {
"objects": articles_detail,
"updates": articles_update,
}
out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
driver.quit() 


yes


In [1]:
import os
import requests
import feedparser
import re
import numpy
from datetime import datetime, timezone
from multiprocessing import Process, set_start_method
import multiprocessing

endpoint = "https://active-monitor-48.hasura.app/v1/graphql"
admin_key = "bAQuK7HSYvMAp6S6pnqXH0wQlyuKNUICzoW3jwecc27pwz6COLhE750s5YAec7Hz"

def query_hasura_graphql(endpoint, admin_key, query, variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }

    data = {
        'query': query,
        'variables': variables
    }
    response = requests.post(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)

        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False

def mutation_hasura_graphql(endpoint, admin_key, mutation_query, mutation_variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }
    response = requests.post(endpoint, json={'query': mutation_query, 'variables': mutation_variables}, headers=headers)
    if response.ok:
        data = response.json()
        print(data)
        return True, data
    else:
        print(f"Mutation failed with status code {response.status_code}: {response.text}")
        return False, None

In [6]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
  synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}, article_id: {_eq: "28057"}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
    article_id
    initial_group
  }
}
'''
query2 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
    article_id
    initial_group
    }
}
'''
query3 = '''
query MyQuery($articleid: [bigint!] = []) {
    synopse_articles_t_v3_article_groups_l2(where: {articles_group: {_contains: $articleid}}, order_by: {is_valid: desc, updated_at: desc}) {
    id
    articles_group
    is_valid
    }
}
'''
query4 = '''
query MyQuery($articleid: [bigint!] = []) {
  synopse_articles_t_v3_article_groups_l2(order_by: {updated_at: desc}, where: {is_valid: {_eq: 1}, articles_group: {_contains: $articleid}}, limit: 1) {
    articles_group
    id
  }
}
'''
mutation_query = """
mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}, $articleGroupIds: [bigint!] = "") {
  insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
    affected_rows
  }
  update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
    affected_rows
  }
  update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
    affected_rows
  }
  delete_synopse_articles_t_v3_article_groups_l2(where: {id: {_in: $articleGroupIds}}) {
    affected_rows
  }
}
"""
variables = {
"limit": 1,
"offset": 0
}
response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
synopse_articles_t_v1_rss1_articles_updates_loc=[]
synopse_articles_t_v3_article_groups_l2_updates_loc=[]
articleGroupIds_loc=[]
print(response_data)
for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
  variables2 = {
      "articleid": [response['article_id']]
      }
  response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
  print(response_data1)
  articles_ids = []
  if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
      for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
          articles_ids.append(func_response['initial_group'])
  n1 = []
  for sublist in articles_ids:
      for element in sublist:
          n1.append(element)
  articles_ids = list(set(n1))
  articles_ids.sort(reverse=True)
  print(articles_ids)
  articles_valid = []
  while True:
      if len(articles_ids) > 1:
          articles_news = []
          for article_id in articles_ids:
              variables3 = {
                  "articleid": [article_id]
                  }
              response_data2 = query_hasura_graphql(endpoint, admin_key, query2, variables3)
              if len(response_data2['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                  for func_response in response_data2['data']['synopse_articles_t_v3_article_groups_l1']:
                      articles_news.append(func_response['initial_group'])
          n2 = []
          for sublist in articles_news:
              for element in sublist:
                  n2.append(element)
          articles_news = list(set(n2))
          articles_news.sort(reverse=True)
          if articles_ids == articles_news:
              break
          else:
              articles_ids = articles_news
  print(articles_ids)

{'data': {'synopse_articles_t_v3_article_groups_l1': [{'article_id': 28057, 'initial_group': [119, 45904, 38175, 28057]}]}}
{'data': {'synopse_articles_t_v3_article_groups_l1': [{'article_id': 45904, 'initial_group': [443, 28148, 1438, 45904, 38175, 45833, 28057, 1935]}, {'article_id': 28057, 'initial_group': [119, 45904, 38175, 28057]}, {'article_id': 38175, 'initial_group': [119, 45904, 38175, 28057]}]}}
[45904, 45833, 38175, 28148, 28057, 1935, 1438, 443, 119]
[46718, 45904, 45833, 41587, 40592, 38175, 32355, 28148, 28057, 27951, 27005, 25909, 25823, 6854, 6852, 6838, 6836, 6825, 6654, 6307, 3849, 2804, 2773, 2742, 2740, 2738, 2736, 2690, 2672, 2527, 2469, 2467, 2455, 2452, 2445, 2373, 1935, 1922, 1859, 1831, 1825, 1824, 1819, 1815, 1780, 1516, 1511, 1499, 1493, 1489, 1487, 1486, 1459, 1447, 1438, 1429, 1420, 1417, 1409, 1397, 1396, 1394, 1380, 1377, 1375, 1361, 1358, 1335, 500, 449, 443, 403, 349, 347, 331, 319, 316, 305, 296, 285, 284, 283, 129, 122, 119, 116, 111, 109, 108, 94, 8

In [13]:
query5 = '''
query MyQuery($articleid: [bigint!] = []) {
  synopse_articles_t_v3_article_groups_l2(order_by: {updated_at: desc}, where: {is_valid: {_eq: 0}, articles_group: {_contains: $articleid}}, limit: 1) {
    articles_group
    id
  }
}
'''
query4 = '''
query MyQuery($articleid: [bigint!] = []) {
  synopse_articles_t_v3_article_groups_l2(order_by: {updated_at: desc}, where: {is_valid: {_eq: 1}, articles_group: {_contains: $articleid}}, limit: 1) {
    articles_group
    id
    t_v4_article_groups_l2_detail {
      article_comment_id
    }
  }
}
'''
insert = True

articles_ids = list(set(n2))
articles_ids.sort(reverse=True)
variables3 = {
    "articleid": [articles_ids[-1]]
    }
response_data10 = query_hasura_graphql(endpoint, admin_key, query5, variables3)
if len(response_data10['data']['synopse_articles_t_v3_article_groups_l2']) == 1:
    insert = False
    synopse_articles_t_v3_article_groups_l2_updates_loc.append({
                    "where": {"id" : { "_eq": response_data10['data']['synopse_articles_t_v3_article_groups_l2'][0]["id"] }},
                    "_set": {"articles_group": articles_ids, 
                             'articles_in_group': len(articles_ids)}
                    })
else:
    response_data11 = query_hasura_graphql(endpoint, admin_key, query4, variables3)
    if len(response_data11['data']['synopse_articles_t_v3_article_groups_l2']) == 1:
        insert = True
        synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
                "articles_group": articles_ids,
                'articles_in_group': len(articles_ids),
                "article_comment_id" : response_data11['data']['synopse_articles_t_v3_article_groups_l2'][0]["t_v4_article_groups_l2_detail"]["article_comment_id"]
                }
            )
    else:
        insert = True
        synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
                "articles_group": articles_ids,
                'articles_in_group': len(articles_ids),
                "article_comment_id" : articles_ids[-1]
                }
            )
    
for article in articles_ids:
    synopse_articles_t_v1_rss1_articles_updates_loc.append({
        "where": {"id" : { "_eq": article }},
        "_set": {"is_grouped": 2}
        })
print(synopse_articles_t_v3_article_groups_l2_insert_input_loc)
print(synopse_articles_t_v1_rss1_articles_updates_loc)
print(synopse_articles_t_v3_article_groups_l2_updates_loc)

{'articleid': [75]}
{'data': {'synopse_articles_t_v3_article_groups_l2': []}}
{'data': {'synopse_articles_t_v3_article_groups_l2': [{'articles_group': [1516, 1511, 1499, 1493, 1489, 1487, 1486, 1459, 1447, 1438, 1429, 1420, 1417, 1409, 1397, 1396, 1394, 1380, 1377, 1375, 1361, 1358, 1335, 500, 449, 443, 403, 349, 331, 305, 296, 285, 284, 283, 116, 111, 109, 108, 94, 75], 'id': 1163, 't_v4_article_groups_l2_detail': {'article_comment_id': 1163}}]}}
[{'articles_group': [46718, 45904, 45833, 41587, 40592, 38175, 32355, 28148, 28057, 27951, 27005, 25909, 25823, 6854, 6852, 6838, 6836, 6825, 6654, 6307, 3849, 2804, 2773, 2742, 2740, 2738, 2736, 2690, 2672, 2527, 2469, 2467, 2455, 2452, 2445, 2373, 1935, 1922, 1859, 1831, 1825, 1824, 1819, 1815, 1780, 1516, 1511, 1499, 1493, 1489, 1487, 1486, 1459, 1447, 1438, 1429, 1420, 1417, 1409, 1397, 1396, 1394, 1380, 1377, 1375, 1361, 1358, 1335, 500, 449, 443, 403, 349, 347, 331, 319, 316, 305, 296, 285, 284, 283, 129, 122, 119, 116, 111, 109, 108, 9

In [2]:
import os
import requests
import feedparser
import re
import numpy
from datetime import datetime, timezone
from multiprocessing import Process, set_start_method
import multiprocessing

endpoint = "https://active-monitor-48.hasura.app/v1/graphql"
admin_key = "bAQuK7HSYvMAp6S6pnqXH0wQlyuKNUICzoW3jwecc27pwz6COLhE750s5YAec7Hz"

def query_hasura_graphql(endpoint, admin_key, query, variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }

    data = {
        'query': query,
        'variables': variables
    }
    response = requests.post(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)

        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False

def mutation_hasura_graphql(endpoint, admin_key, mutation_query, mutation_variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }
    response = requests.post(endpoint, json={'query': mutation_query, 'variables': mutation_variables}, headers=headers)
    if response.ok:
        data = response.json()
        print(data)
        return True, data
    else:
        print(f"Mutation failed with status code {response.status_code}: {response.text}")
        return False, None

def grouping_l2(offset1):
    graphql_query = '''
    query MyQuery($limit: Int!, $offset: Int!) {
      synopse_articles_t_v3_article_groups_l1(where: {t_v1_rss1_article: {is_grouped: {_eq: 1}}, article_count: {_gt: 1}}, order_by: {updated_at: asc}, limit: $limit, offset: $offset) {
        article_id
        initial_group
      }
    }
    '''
    query2 = '''
    query MyQuery($articleid: [bigint!] = []) {
        synopse_articles_t_v3_article_groups_l1(where: {initial_group: {_contains: $articleid}}) {
        article_id
        initial_group
        }
    }
    '''
    query5 = '''
    query MyQuery($articleid: [bigint!] = []) {
      synopse_articles_t_v3_article_groups_l2(order_by: {updated_at: desc}, where: {is_valid: {_eq: 0}, articles_group: {_contains: $articleid}}, limit: 1) {
        articles_group
        id
      }
    }
    '''
    query4 = '''
    query MyQuery($articleid: [bigint!] = []) {
      synopse_articles_t_v3_article_groups_l2(order_by: {updated_at: desc}, where: {is_valid: {_eq: 1}, articles_group: {_contains: $articleid}}, limit: 1) {
        articles_group
        id
        article_comment_id
      }
    }
    '''
    mutation_query = """
    mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l2_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}, $updates1: [synopse_articles_t_v3_article_groups_l2_updates!] = {where: {}}, $articleGroupIds: [bigint!] = "") {
      insert_synopse_articles_t_v3_article_groups_l2(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l2_pkey}) {
        affected_rows
      }
      update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
        affected_rows
      }
      update_synopse_articles_t_v3_article_groups_l2_many(updates: $updates1) {
        affected_rows
      }
      delete_synopse_articles_t_v3_article_groups_l2(where: {id: {_in: $articleGroupIds}}) {
        affected_rows
      }
    }
    """
    while True:
        variables = {
        "limit": 1,
        "offset": 0
        }
        response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
        if len(response_data['data']['synopse_articles_t_v3_article_groups_l1']) == 0:
            break
        synopse_articles_t_v3_article_groups_l2_insert_input_loc=[]
        synopse_articles_t_v1_rss1_articles_updates_loc=[]
        synopse_articles_t_v3_article_groups_l2_updates_loc=[]
        articleGroupIds_loc=[]
        for response in response_data['data']['synopse_articles_t_v3_article_groups_l1']:
            variables2 = {
                "articleid": [response['article_id']]
                }
            response_data1 = query_hasura_graphql(endpoint, admin_key, query2, variables2)
            articles_ids = []
            if len(response_data1['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                for func_response in response_data1['data']['synopse_articles_t_v3_article_groups_l1']:
                    articles_ids.append(func_response['initial_group'])
            n1 = []
            for sublist in articles_ids:
                for element in sublist:
                    n1.append(element)
            articles_ids = list(set(n1))
            articles_ids.sort(reverse=True)
            articles_valid = []
            while True:
                if len(articles_ids) > 1:
                    articles_news = []
                    for article_id in articles_ids:
                        variables3 = {
                            "articleid": [article_id]
                            }
                        response_data2 = query_hasura_graphql(endpoint, admin_key, query2, variables3)
                        if len(response_data2['data']['synopse_articles_t_v3_article_groups_l1']) > 0:
                            for func_response in response_data2['data']['synopse_articles_t_v3_article_groups_l1']:
                                articles_news.append(func_response['initial_group'])
                    n2 = []
                    for sublist in articles_news:
                        for element in sublist:
                            n2.append(element)
                    articles_news = list(set(n2))
                    articles_news.sort(reverse=True)
                    if articles_ids == articles_news:
                        break
                    else:
                        articles_ids = articles_news
            print(articles_ids)
            insert = True

            articles_ids = list(set(n2))
            articles_ids.sort(reverse=True)
            variables3 = {
                "articleid": [articles_ids[-1]]
                }
            response_data10 = query_hasura_graphql(endpoint, admin_key, query5, variables3)
            if len(response_data10['data']['synopse_articles_t_v3_article_groups_l2']) == 1:
                insert = False
                synopse_articles_t_v3_article_groups_l2_updates_loc.append({
                                "where": {"id" : { "_eq": response_data10['data']['synopse_articles_t_v3_article_groups_l2'][0]["id"] }},
                                "_set": {"articles_group": articles_ids, 
                                        'articles_in_group': len(articles_ids)}
                                })
            else:
                response_data11 = query_hasura_graphql(endpoint, admin_key, query4, variables3)
                if len(response_data11['data']['synopse_articles_t_v3_article_groups_l2']) == 1:
                    insert = True
                    synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
                            "articles_group": articles_ids,
                            'articles_in_group': len(articles_ids),
                            "article_comment_id" : response_data11['data']['synopse_articles_t_v3_article_groups_l2'][0]["article_comment_id"]
                            }
                        )
                else:
                    insert = True
                    synopse_articles_t_v3_article_groups_l2_insert_input_loc.append({
                            "articles_group": articles_ids,
                            'articles_in_group': len(articles_ids),
                            "article_comment_id" : articles_ids[-1]
                            }
                        )
                
            for article in articles_ids:
                synopse_articles_t_v1_rss1_articles_updates_loc.append({
                    "where": {"id" : { "_eq": article }},
                    "_set": {"is_grouped": 2}
                    })
            mutation_variables = {
                "objects": synopse_articles_t_v3_article_groups_l2_insert_input_loc,
                "updates": synopse_articles_t_v1_rss1_articles_updates_loc,
                "updates1": synopse_articles_t_v3_article_groups_l2_updates_loc,
                "articleGroupIds": articleGroupIds_loc
                }
            out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
        
        


grouping_l2(0)